In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras import layers, Model
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, classification_report

# ===== MOUNT GOOGLE DRIVE (if using Colab) =====
from google.colab import drive
drive.mount('/content/drive')

# ===== DATASET PATH =====
dataset_path = "/content/drive/MyDrive/NNDL Dataset/Dataset"

# ===== CATEGORIES =====
categories = ["gond painting", "kalighat painting", "kangra painting", "kerala mural",
              "madhubani painting", "mandana art drawing", "pichwai painting", "warli painting"]

# ===== IMAGE LOADING SETTINGS =====
img_size = (64, 64)  # Reduce image size
max_images_per_class = 200  # Limit per class

def load_images(folder_path, img_size, max_images):
    images, labels = [], []
    for category in categories:
        category_path = os.path.join(folder_path, category)
        if not os.path.exists(category_path):
            continue
        files = [f for f in os.listdir(category_path) if f.lower().endswith((".jpg", ".jpeg", ".png"))][:max_images]
        for filename in files:
            img_path = os.path.join(category_path, filename)
            img = load_img(img_path, target_size=img_size)
            img_array = img_to_array(img) / 255.0  # Normalize
            images.append(img_array)
            labels.append(categories.index(category))
    return np.array(images), np.array(labels)

# ===== LOAD DATASET =====
X, y = load_images(dataset_path, img_size, max_images_per_class)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# ===== DEFINE CNN MODEL =====
def create_cnn(input_shape):
    inputs = layers.Input(shape=input_shape)
    x = layers.Conv2D(32, (3, 3), activation='relu')(inputs)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Conv2D(64, (3, 3), activation='relu')(x)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Flatten()(x)
    feature_output = layers.Dense(128, activation='relu', name='feature_layer')(x)
    x = layers.Dense(len(categories), activation='softmax')(feature_output)
    return Model(inputs=inputs, outputs=x)

# ===== OPTIMIZERS TO TEST =====
optimizers = {
    "Adam": tf.keras.optimizers.Adam(),
    "SGD": tf.keras.optimizers.SGD(),
    "RMSprop": tf.keras.optimizers.RMSprop()
}

# ===== CLASSIFIERS =====
classifiers = {
    "kNN": KNeighborsClassifier(n_neighbors=3),
    "Decision Tree": DecisionTreeClassifier(),
    "Random Forest": RandomForestClassifier(n_estimators=50),
    "SVM": SVC(kernel='linear'),
    "Naïve Bayes": GaussianNB()
}

# ===== TRAIN & EVALUATE CNN FOR EACH OPTIMIZER =====
results = {}
early_stopping = EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)

for opt_name, optimizer in optimizers.items():
    print(f"\n🔹 Training with {opt_name} optimizer...")

    # Train CNN
    cnn_model = create_cnn((64, 64, 3))
    cnn_model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    history = cnn_model.fit(X_train, y_train, epochs=5, validation_data=(X_val, y_val), batch_size=16, callbacks=[early_stopping], verbose=1)

    # Store CNN loss & accuracy
    cnn_loss = history.history['val_loss'][-1]
    cnn_accuracy = history.history['val_accuracy'][-1]

    # Extract features
    feature_extractor = Model(inputs=cnn_model.input, outputs=cnn_model.get_layer('feature_layer').output)
    X_train_features = feature_extractor.predict(X_train, batch_size=32)
    X_val_features = feature_extractor.predict(X_val, batch_size=32)

    # Train classifiers & store metrics
    classifier_results = {}
    for clf_name, clf in classifiers.items():
        clf.fit(X_train_features, y_train)
        y_pred = clf.predict(X_val_features)

        # Compute metrics
        accuracy = accuracy_score(y_val, y_pred)
        report = classification_report(y_val, y_pred, output_dict=True)

        precision = report["weighted avg"]["precision"]
        recall = report["weighted avg"]["recall"]
        f1_score = report["weighted avg"]["f1-score"]

        classifier_results[clf_name] = {
            "Accuracy": accuracy,
            "Precision": precision,
            "Recall": recall,
            "F1-Score": f1_score
        }

    results[opt_name] = {
        "CNN Loss": cnn_loss,
        "CNN Accuracy": cnn_accuracy,
        "Classifiers": classifier_results
    }

# ===== PRINT RESULTS =====
for opt, res in results.items():
    print(f"\n🔹 Results for {opt} Optimizer:")
    print(f"   ▶ CNN Loss: {res['CNN Loss']:.4f}")
    print(f"   ▶ CNN Accuracy: {res['CNN Accuracy']:.4f}")
    for clf_name, metrics in res["Classifiers"].items():
        print(f"   ▶ {clf_name}: Accuracy = {metrics['Accuracy']:.4f}, Precision = {metrics['Precision']:.4f}, Recall = {metrics['Recall']:.4f}, F1-Score = {metrics['F1-Score']:.4f}")


Mounted at /content/drive


/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1045: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(



🔹 Training with Adam optimizer...
Epoch 1/5
45/45 ━━━━━━━━━━━━━━━━━━━━ 8s 133ms/step - accuracy: 0.1971 - loss: 2.1569 - val_accuracy: 0.3315 - val_loss: 1.9594
Epoch 2/5
45/45 ━━━━━━━━━━━━━━━━━━━━ 9s 103ms/step - accuracy: 0.3214 - loss: 1.7992 - val_accuracy: 0.2753 - val_loss: 1.8372
Epoch 3/5
45/45 ━━━━━━━━━━━━━━━━━━━━ 6s 123ms/step - accuracy: 0.4484 - loss: 1.5732 - val_accuracy: 0.4438 - val_loss: 1.7231
Epoch 4/5
45/45 ━━━━━━━━━━━━━━━━━━━━ 10s 113ms/step - accuracy: 0.5983 - loss: 1.2502 - val_accuracy: 0.4326 - val_loss: 1.6440
Epoch 5/5
45/45 ━━━━━━━━━━━━━━━━━━━━ 5s 115ms/step - accuracy: 0.6697 - loss: 0.9920 - val_accuracy: 0.4551 - val_loss: 1.6889
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 44ms/step
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step

🔹 Training with SGD optimizer...
Epoch 1/5


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


45/45 ━━━━━━━━━━━━━━━━━━━━ 6s 128ms/step - accuracy: 0.1774 - loss: 2.0684 - val_accuracy: 0.2247 - val_loss: 2.0492
Epoch 2/5
45/45 ━━━━━━━━━━━━━━━━━━━━ 8s 89ms/step - accuracy: 0.2664 - loss: 1.9630 - val_accuracy: 0.1573 - val_loss: 2.0093
Epoch 3/5
45/45 ━━━━━━━━━━━━━━━━━━━━ 6s 110ms/step - accuracy: 0.2834 - loss: 1.9252 - val_accuracy: 0.1404 - val_loss: 1.9739
Epoch 4/5
45/45 ━━━━━━━━━━━━━━━━━━━━ 4s 88ms/step - accuracy: 0.2977 - loss: 1.8596 - val_accuracy: 0.3034 - val_loss: 1.8384
Epoch 5/5
45/45 ━━━━━━━━━━━━━━━━━━━━ 4s 99ms/step - accuracy: 0.3479 - loss: 1.7697 - val_accuracy: 0.3090 - val_loss: 1.8160
23/23 ━━━━━━━━━━━━━━━━━━━━ 2s 73ms/step
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step

🔹 Training with RMSprop optimizer...
Epoch 1/5
45/45 ━━━━━━━━━━━━━━━━━━━━ 6s 98ms/step - accuracy: 0.2113 - loss: 2.4315 - val_accuracy: 0.2640 - val_loss: 2.1117
Epoch 2/5
45/45 ━━━━━━━━━━━━━━━━━━━━ 5s 118ms/step - accuracy: 0.2825 - loss: 1.8889 - val_accuracy: 0.3596 - val_loss: 1.7814
Epoch 3/5